In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import scipy.signal
import time
import cv2
import matplotlib.pyplot as plt

In [2]:
tf.config.list_physical_devices("GPU")

[]

In [3]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.InteractiveSession(config=config)

In [4]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [5]:
num_actions = 3
observation_dimensions = (128, 128, 3)

In [6]:
def Model(acc_actions, steer_actions):
    inp = keras.layers.Input( shape=observation_dimensions )
    x = keras.layers.Conv2D(filters=32, kernel_size=(3,3), kernel_initializer='he_normal', 
                            padding='same', activation="relu")(inp)
    
    x = keras.layers.AveragePooling2D( (2,2) )(x)
    x = keras.layers.Conv2D(filters=8, kernel_size=(3,3), kernel_initializer='he_normal', 
                            padding='same', activation="relu")(x)
    
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    
    x1 = keras.layers.Dense(64, activation="relu")(x)
    x1 = keras.layers.Dense(acc_actions, activation="softmax", name="throttle")(x1)
    
    x2 = keras.layers.Dense(64, activation="relu")(x)
    x2 = keras.layers.Dense(steer_actions, activation="softmax", name='steer')(x2)
    
    return keras.models.Model( inp, [x1, x2] )

In [7]:
model = Model(2, 3)
model.compile( optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 64, 64, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 8)    2312        average_pooling2d[0][0]          
______________________________________________________________________________________________

In [8]:
import os

In [9]:
data = pd.read_csv("action_took.csv").drop_duplicates()
print(data.shape)
data = data[ data.iloc[:,0] != "0" ]
data.shape

(1791, 3)


(1790, 3)

In [10]:
up_down_target = data.iloc[:,1]
up_down_target.iloc[data.iloc[:,1] == -1] = 0
up_down_target.value_counts()

/opt/anaconda3/envs/c557/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


1    1639
0     151
Name: 1, dtype: int64

In [11]:
left_right_target = data.iloc[:,2]
left_right_target.value_counts()

 1    729
 0    680
-1    381
Name: 2, dtype: int64

In [12]:
images = []
for img in data.iloc[:,0]:
    images.append(cv2.imread(img).reshape((-1,128,128,3)) )
images = np.concatenate(images, axis=0)
images.shape

(1790, 128, 128, 3)

In [13]:
up_down_target = (up_down_target).values
left_right_target = (left_right_target+1).values

In [14]:
target_ud = np.zeros( (len(images), 2) )
target_lr = np.zeros( (len(images), 3) )
target_ud.shape, target_lr.shape

((1790, 2), (1790, 3))

In [15]:
for i in range(len(up_down_target)):
    target_ud[i][ up_down_target[i] ] = 1
    
for i in range(len(left_right_target)):
    target_lr[i][ left_right_target[i] ] = 1

In [16]:
target_ud.sum(axis=0), target_lr.sum(axis=0)

(array([ 151., 1639.]), array([381., 680., 729.]))

In [17]:
history = model.fit(images/255, [target_ud, target_lr], validation_split=0.2, batch_size=128, epochs=5)

Epoch 1/5
12/12 [==============================] - 15s 1s/step - loss: 1.2736 - throttle_loss: 0.3777 - steer_loss: 0.8958 - throttle_accuracy: 0.9239 - steer_accuracy: 0.6697 - val_loss: 0.8430 - val_throttle_loss: 0.0814 - val_steer_loss: 0.7616 - val_throttle_accuracy: 0.9916 - val_steer_accuracy: 0.6285
Epoch 2/5
12/12 [==============================] - 11s 920ms/step - loss: 0.4281 - throttle_loss: 0.0764 - steer_loss: 0.3518 - throttle_accuracy: 0.9791 - steer_accuracy: 0.8547 - val_loss: 0.6506 - val_throttle_loss: 0.0405 - val_steer_loss: 0.6101 - val_throttle_accuracy: 0.9916 - val_steer_accuracy: 0.6955
Epoch 3/5
12/12 [==============================] - 11s 918ms/step - loss: 0.2673 - throttle_loss: 0.0302 - steer_loss: 0.2371 - throttle_accuracy: 0.9916 - steer_accuracy: 0.9022 - val_loss: 0.8053 - val_throttle_loss: 0.0307 - val_steer_loss: 0.7746 - val_throttle_accuracy: 0.9944 - val_steer_accuracy: 0.6425
Epoch 4/5
12/12 [==============================] - 10s 813ms/step -

In [18]:
import gym
import gym_carla
import carla 

In [19]:
#WetCloudyNoon, WetSunset, HardRainNoon

In [20]:
params = {
    'number_of_vehicles': 40,
    'number_of_walkers': 30,
    'display_size': 250,  # screen size of bird-eye render
    'display_height' : 512,
    'display_main': True,
    'weather': "WetSunset",
    'max_past_step': 1,  # the number of past steps to draw
    'dt': 0.1,  # time interval between two frames
    'discrete': False,  # whether to use discrete control space
    'discrete_acc': [1.0, 0.0, 1.0],  # discrete value of accelerations
    'discrete_steer': [-1, 0, 1],  # discrete value of steering angles
    'continuous_accel_range': [-3.0, 3.0],  # continuous acceleration range
    'continuous_steer_range': [-0.2, 0.2],  # continuous steering angle range
    'ego_vehicle_filter': 'vehicle.tesla.model3',  # filter for defining ego vehicle
    'address': 'localhost',
    'port': 2000, # connection port
    'town': 'Town02', # which town to simulate
    'task_mode': 'random',  # mode of the task, [random, roundabout (only for Town03)]
    'max_time_episode': 5000,  # maximum timesteps per episode
    'max_waypt': 12,  # maximum number of waypoints
    'obs_range': 32,  # observation range (meter)
    'lidar_bin': 0.125,  # bin size of lidar sensor (meter)
    'd_behind': 12,  # distance behind the ego vehicle (meter)
    'out_lane_thres': 5.0,  # threshold for out of lane
    'desired_speed': 8,  # desired speed (m/s)
    'max_ego_spawn_times': 200,  # maximum times to spawn ego vehicle
    'display_route': True,  # whether to render the desired route50
    'pixor_size': 64,  # size of the pixor labels
    'pixor': False,  # whether to output PIXOR observation
  }

In [21]:
def read_transform(img):
    return img[76:204,76:204,:]/255
env = gym.make('carla-v0', params=params)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
connecting to Carla server...
Carla server connected!


In [25]:
observation =  env.reset()
for _ in range(20): observation, _, _, _ = env.step([1.25,0])
done = False
while not done:
    #action_ud = np.argmax(up_down_model.predict( read_transform(observation['birdeye']).reshape( (1, 128,128,3) ))[0])
    #action_lr = np.argmax(left_right_model.predict( read_transform(observation['birdeye']).reshape( (1, 128,128,3) ))[0])-1
    action_ud, action_lr = model.predict( read_transform(observation['birdeye']).reshape( (1, 128,128,3) ))
    action_ud, action_lr = np.argmax(action_ud[0]), np.argmax(action_lr[0])-1
    observation, _, done, _ = env.step( [1.25*action_ud if action_ud == 1 else -1, action_lr] )